In [1]:
import opensim
import math
import numpy as np
import sys
import os
from osim.env import OsimEnv
import os.path

In [2]:
from GaitEnv import GaitEnv
from BioInspiredHierarchicalLearning import BioInspiredHierarchicalLearning

In [3]:
class TwitchingEnv(GaitEnv):
    def __init__(self, visualize = False, musclesName = '', iterDuration = 1, twitchDurationPercentage = 10, stepsize = 0.01):
        print("Parameters copy...")
        self.model_path = './gait9dof18musc.osim'
        folder = ''
        self.musclesName = musclesName
        
        #what does this mean ?
        self.iterNumber = 14;
        self.iterDuration = iterDuration-stepsize
        self.twitchDurationPercentage = twitchDurationPercentage
        self.twitchingEpisodeStartedAt = 0;
        self.twitchStartPercentage = 20;

        self.twitchingEpisodeEnded = True
        
        print("Files loading...")
        self.sensorsFile = open('sensors.csv', 'w')
        self.actuatorsFile = open('actuators.csv', 'w')

        self.musclesLength = [ 0 for i in range(len(musclesName))]
        self.musclesLengthConverged = [ 0 for i in range(len(musclesName))]

        print("Headers wirting...")

        self.writeHeaders([m + '_len' for m in self.musclesName] + [m + '_dlen' for m in self.musclesName],self.sensorsFile)
        self.writeHeaders([m + '_a' for m in self.musclesName],self.actuatorsFile)
        
        print("GaitEnv __init__ calling...")
        super(TwitchingEnv, self).__init__(visualize = visualize, stepsize = stepsize)
    def resetHanged(self):
        self.reset();
        #import ipdb; ipdb.set_trace()
        self.osim_model.model.updCoordinateSet().get('pelvis_tilt').setLocked(self.osim_model.state, True)
        self.osim_model.model.updCoordinateSet().get('pelvis_tx').setLocked(self.osim_model.state, True)
        self.osim_model.model.updCoordinateSet().get('pelvis_ty').setLocked(self.osim_model.state, True)
        self.osim_model.model.updCoordinateSet().get('lumbar_extension').setLocked(self.osim_model.state, True)
        
    def get_observation(self):
        super(TwitchingEnv, self).get_observation()

        m1_length = self.osim_model.model.getActuators().get(1).getStateVariableValues(self.osim_model.state).get(1);
        m1_name = self.osim_model.model.getActuators().get(1).getStateVariableNames().get(1)

    def writeHeaders(self,variable_names,file):
        file.write("{}\n".format(",".join(variable_names)))
    def writeContent(self,variables,file):
        file.write("{}\n".format(",".join([str(x) for x in variables])))

    def activate_muscles(self, action, twitch, muscleNum):
        muscleSet = self.osim_model.model.getMuscles()
        for j in range(self.noutput):
            muscle = muscleSet.get(j)
            if j == muscleNum and twitch == True:
                muscle.setActivation(self.osim_model.state, 0.99)

    def _step(self, action):
        self.last_action = action

        # TODO: Make learning on matlab 
        # TODO: Implement the reflex controller
        # TODO: Modify the get_observation to include muscle length information.
        totalTime = self.istep*self.stepsize
        time = math.fmod(totalTime,self.iterDuration)


        # Twitching experiment procedure
        twitchingStartEpisode = False
        if self.twitchingEpisodeEnded:
            print("-- start next episode")
            self.twitchingEpisodeStartedAt = self.istep*self.stepsize
            self.twitchingEpisodeEnded = False
            twitchingStartEpisode = True

        if (twitchingStartEpisode):
            self.musclesLengthConverged = self.musclesLength
            self.resetHanged()
            #self.iterNumber = self.iterNumber+1;
            print("Twitching muscle number {}".format(self.iterNumber))
        


        # Actual Twitches
        if( time > self.iterDuration*self.twitchStartPercentage/100.0 
            and
            time <= self.iterDuration*(self.twitchDurationPercentage+self.twitchStartPercentage)/100.):
            self.activate_muscles(action,True,self.iterNumber )
        else:
            self.activate_muscles(action,False,self.iterNumber)

        
        if (self.istep*self.stepsize >= self.iterDuration):
            print("-- episode ended")
            self.twitchingEpisodeEnded = True;



        # Get input / outputs from open sim
        musclesLength_new = [self.osim_model.model.getActuators().get(m).getStateVariableValues(self.osim_model.state).get(1) for m in self.musclesName]
        musclesdLength = [ (x-y)/self.stepsize for x,y in zip( musclesLength_new, self.musclesLength) ]
        actuatorsValues = [self.osim_model.model.getActuators().get(m).getStateVariableValues(self.osim_model.state).get(0) for m in self.musclesName]


        sensorsValues = musclesLength_new + musclesdLength 

        if len(musclesdLength) == 0:
            sensorsValues = sensorsValues + [ 0 for i in range(len(self.musclesName))]

        if time <= self.iterDuration*self.twitchStartPercentage/100.0:
            #print("{}, {} <= {} ===> {}".format(self.iterDuration,time,self.iterDuration*self.twitchStartPercentage/100.0,time <= self.iterDuration*self.twitchStartPercentage/100.0))
            sensorsValues = self.musclesLengthConverged + [ 0 for i in range(len(self.musclesName))]

        self.musclesLength = musclesLength_new

        # Logging
        self.writeContent(sensorsValues,self.sensorsFile)
        self.writeContent(actuatorsValues,self.actuatorsFile)

        # Integrate one step
        self.osim_model.manager.setInitialTime(self.stepsize * self.istep)
        self.osim_model.manager.setFinalTime(self.stepsize * (self.istep + 1))


        try:
            self.osim_model.manager.integrate(self.osim_model.state)
        except Exception:
            print ("Exception raised")
            return self.get_observation(), -500, True, {}

        self.istep = self.istep + 1
        TT = self.osim_model.model.getActuators().get(1)

        res = [ self.get_observation(), self.compute_reward(), self.is_done(), {} ]
        
        return res






In [4]:
print("Beginning")

# TODO ? minutes : Increase simulation duration time to M*T = 9*500
muscle_names = [
				"hamstrings_l",
				"bifemsh_l",
				"glut_max_l",
				"iliopsoas_l",
				"rect_fem_l",
				"vasti_l",
				"gastroc_l",
				"soleus_l",
				"tib_ant_l",
				"hamstrings_r",
				"bifemsh_r",
				"glut_max_r",
				"iliopsoas_r",
				"rect_fem_r",
				"vasti_r",
				"gastroc_r",
				"soleus_r",
				"tib_ant_r"
				]

twitchDuration = 0.05 # seconds
iterationDuration = 1 # seconds

twitchDurationPercentage = int(twitchDuration/5*100)







Beginning


In [5]:
print("Environment Generation")
env = TwitchingEnv(visualize=False, musclesName = muscle_names, iterDuration=iterationDuration, 
                   twitchDurationPercentage=twitchDurationPercentage, stepsize = 0.01)

Environment Generation
Parameters copy...
Files loading...
Headers wirting...
GaitEnv __init__ calling...
pelvis
femur_r
tibia_r
talus_r
calcn_r
toes_r
femur_l
tibia_l
talus_l
calcn_l
toes_l
torso
head


In [6]:
env.resetHanged()

In [42]:
print(env.action_space.shape)
print(env.current_state.shape)
print([3]*24)

(18,)
(31,)
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [7]:

duration = len(env.musclesName)*(env.iterDuration);

print("Twitching Experiment")

for i in range(int(1.0/env.stepsize*duration)):
	percentage = int(1000*i/(1.0/env.stepsize*duration-1))/10.0;

	print("{}%".format(percentage))
	percentage = math.fmod(i,duration/env.stepsize/100)

	env.step([i] * 24)

Twitching Experiment
0.0%
-- start next episode
Twitching muscle number 14
0.0%
0.1%
0.1%
0.2%
0.2%
0.3%
0.3%
0.4%
0.5%
0.5%
0.6%
0.6%
0.7%
0.7%
0.8%
0.8%
0.9%
1.0%
1.0%
1.1%
1.1%
1.2%
1.2%
1.3%
1.4%
1.4%
1.5%
1.5%
1.6%
1.6%
1.7%
1.7%
1.8%
1.9%
1.9%
2.0%
2.0%
2.1%
2.1%
2.2%
2.3%
2.3%
2.4%
2.4%
2.5%
2.5%
2.6%
2.6%
2.7%
2.8%
2.8%
2.9%
2.9%
3.0%
3.0%
3.1%
3.2%
3.2%
3.3%
3.3%
3.4%
3.4%
3.5%
3.5%
3.6%
3.7%
3.7%
3.8%
3.8%
3.9%
3.9%
4.0%
4.0%
4.1%
4.2%
4.2%
4.3%
4.3%
4.4%
4.4%
4.5%
4.6%
4.6%
4.7%
4.7%
4.8%
4.8%
4.9%
4.9%
5.0%
5.1%
5.1%
5.2%
5.2%
5.3%
5.3%
5.4%
5.5%
5.5%
-- episode ended
5.6%
-- start next episode
Twitching muscle number 14
5.6%
5.7%
5.7%
5.8%
5.8%
5.9%
6.0%
6.0%
6.1%
6.1%
6.2%
6.2%
6.3%
6.4%
6.4%
6.5%
6.5%
6.6%
6.6%
6.7%
6.7%
6.8%
6.9%
6.9%
7.0%
7.0%
7.1%
7.1%
7.2%
7.2%
7.3%
7.4%
7.4%
7.5%
7.5%
7.6%
7.6%
7.7%
7.8%
7.8%
7.9%
7.9%
8.0%
8.0%
8.1%
8.1%
8.2%
8.3%
8.3%
8.4%
8.4%
8.5%
8.5%
8.6%
8.7%
8.7%
8.8%
8.8%
8.9%
8.9%
9.0%
9.0%
9.1%
9.2%
9.2%
9.3%
9.3%
9.4%
9.4%
9.5%
9.6%
9.6%

### Matrix Correlation Computation

In [48]:
# Twitching process
import pandas as pd

sensors = pd.read_csv('sensors.csv');
actuators = pd.read_csv('actuators.csv');
length_0 = pd.read_csv('length_0', header=None).values
print(length_0)
length_0 = length_0[0][1:9]

print(length_0)



[[ 0.06795151  0.13681547  0.15901792  0.1324084   0.05935354  0.07428662
   0.0426583   0.02715707  0.08514987  0.06795151  0.13681547  0.15901792
   0.1324084   0.05935354  0.07428662  0.0426583   0.02715707  0.08514987]]
[ 0.13681547  0.15901792  0.1324084   0.05935354  0.07428662  0.0426583
  0.02715707  0.08514987]


In [11]:
actuators

,hamstrings_l_a,bifemsh_l_a,glut_max_l_a,iliopsoas_l_a,rect_fem_l_a,vasti_l_a,gastroc_l_a,soleus_l_a,tib_ant_l_a,hamstrings_r_a,bifemsh_r_a,glut_max_r_a,iliopsoas_r_a,rect_fem_r_a,vasti_r_a,gastroc_r_a,soleus_r_a,tib_ant_r_a
0,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
1,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680,0.044680
2,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123,0.040123
3,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206,0.036206
4,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829,0.032829
5,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912,0.029912
6,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384,0.027384
7,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191,0.025191
8,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285,0.023285
9,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626,0.021626


In [41]:
sensors

,hamstrings_l_len,bifemsh_l_len,glut_max_l_len,iliopsoas_l_len,rect_fem_l_len,vasti_l_len,gastroc_l_len,soleus_l_len,tib_ant_l_len,hamstrings_r_len,...,tib_ant_l_dlen,hamstrings_r_dlen,bifemsh_r_dlen,glut_max_r_dlen,iliopsoas_r_dlen,rect_fem_r_dlen,vasti_r_dlen,gastroc_r_dlen,soleus_r_dlen,tib_ant_r_dlen
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
print(sensors.shape)
print(actuators.shape)

(1776, 36)
(1774, 18)


In [35]:
# Compute the weights for only one leg
length_0_mat = np.zeros((sensors.values)[:,0:18].shape)

for i in range(length_0_mat.shape[0]):
    length_0_mat[i,:] = length_0

delta_length = (sensors.values)[:,0:18] - length_0_mat
rate_length = (sensors.values)[:,18:36]
stim = actuators.values

weight_1 = np.zeros(9)
weight_2 = np.zeros(9)
all_weight = np.zeros(9)

dt = 0.01
lr = 0.01

In [40]:
def computeActivation(stim, dt):

    past_activation = 0;
    activation = np.zeros(length(stim))

    for i in range(len(stim)):

        if(stim[i] > 0):
            tau = 15;
        else:
            tau = 50;
        
        activation_derivative = tau * ( -past_activation + stim[i] )
        activation[i] = past_activation + dt * activation_derivative
        past_activation = activation[i]
    
    return activation



In [ ]:
or i in range(len(weight_1)):
    
    #% Filter the activation
    ## should we invert line and column ? No
    activation = computeActivation(stim[:,i], dt);
    s = (activation > 0)
    n_s = np.sum(s)
    
    #% 40 replications of a speciffic twitching
    for l in range(40):
        for k in range(n_s):
            for j=1:size(weight_1,2)
                
                % Anti-Oja rule
                weight_1(i,j) = weight_1(i,j) - lr * activation(s(k)) * (delta_length(s(k)+1,j)+activation(s(k))*weight_1(i,j));
                weight_2(i,j) = weight_2(i,j) - lr * activation(s(k)) * (rate_length(s(k)+1,j)+activation(s(k))*weight_2(i,j));
                


% Weights' matrices for both legs
weight_1 =  blkdiag(weight_1,weight_1);
weight_2 =  blkdiag(weight_2,weight_2);

% Saving weights' matrices
dlmwrite('weight_1.csv',weight_1,',');
dlmwrite('weight_2.csv',weight_2,',');



%% Florin's data plot

time = linspace(0,length(actuators.data)/1000,length(actuators.data));

figure;

subplot(311)
plot(time(1:end/2),actuators.data(1:end/2,10:end))
ylabel('stim')
legend(actuators.textdata{10:end})

subplot(312)
plot(time(1:end/2),sensors.data(1:end/2,10:18))
ylabel('II')

subplot(313)
plot(time(1:end/2),sensors.data(1:end/2,18+9:end))
ylabel('Ia')
xlabel('time (s)')

# Twitching process

sensors = importdata('sensors_set4');
actuators = importdata('actuators_set4');
length_0 = importdata('length_0');
length_0 = length_0(1:9);

# Compute the weights for only one leg
delta_length = sensors.data(:,1:9)-repmat(length_0,size(sensors.data,1),1);
rate_length = sensors.data(:,19:28);
stim = actuators.data;

weight_1 = zeros(9);
weight_2 = zeros(9);
all_weight = zeros(9);

dt = 0.01;
lr = 0.01;

for i=1:size(weight_1,1)
    
    % Filter the activation
    activation = computeActivation(stim(:,i), dt);
    s = find(activation > 0);
    
    % 40 replications of a specific twitching
    for l=1:40
        for k=1:length(s)
            for j=1:size(weight_1,2)
                
                % Anti-Oja rule
                weight_1(i,j) = weight_1(i,j) - lr * activation(s(k)) * (delta_length(s(k)+1,j)+activation(s(k))*weight_1(i,j));
                weight_2(i,j) = weight_2(i,j) - lr * activation(s(k)) * (rate_length(s(k)+1,j)+activation(s(k))*weight_2(i,j));
                
            end
        end
    end
end

% Weights' matrices for both legs
weight_1 =  blkdiag(weight_1,weight_1);
weight_2 =  blkdiag(weight_2,weight_2);

% Saving weights' matrices
dlmwrite('weight_1',weight_1,'\t');
dlmwrite('weight_2',weight_2,'\t');



%% Florin's data plot

time = linspace(0,length(actuators.data)/1000,length(actuators.data));

figure;

subplot(311)
plot(time(1:end/2),actuators.data(1:end/2,10:end))
ylabel('stim')
legend(actuators.textdata{10:end})

subplot(312)
plot(time(1:end/2),sensors.data(1:end/2,10:18))
ylabel('II')

subplot(313)
plot(time(1:end/2),sensors.data(1:end/2,18+9:end))
ylabel('Ia')
xlabel('time (s)')